In [5]:
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
import time

In [ ]:
class GridTradingBot(IBapi):
    def __init__(self, symbol, grid_size, quantity):
        super().__init__()
        self.symbol = symbol
        self.grid_size = grid_size
        self.quantity = quantity
        self.positions = []

    def start(self):
        self.connect("127.0.0.1", 7497, 0)
        thread = threading.Thread(target=self.run)
        thread.start()
        time.sleep(1)  # Sleep interval to allow time for connection to server

    def nextValidId(self, orderId: int):
        self.nextOrderId = orderId
        self.place_initial_orders()

    def place_initial_orders(self):
        # Place initial buy and sell orders
        self.place_order(self.symbol, "BUY", self.quantity, self.grid_size)
        self.place_order(self.symbol, "SELL", self.quantity, self.grid_size * 2)

    def place_order(self, symbol, action, quantity, price):
        order = ibapi.order.Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = price
        self.placeOrder(self.nextOrderId, self.create_contract(symbol), order)
        self.nextOrderId += 1

    def create_contract(self, symbol):
        contract = ibapi.contract.Contract()
        contract.symbol = symbol
        contract.secType = "STK"
        contract.exchange = "SMART"
        contract.currency = "USD"
        return contract

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. Id: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, LastFillPrice: {lastFillPrice}")

    def execDetails(self, reqId, contract, execution):
        print(f"ExecDetails. ReqId: {reqId}, Symbol: {contract.symbol}, SecType: {contract.secType}, Currency: {contract.currency}, ExecId: {execution.execId}, OrderId: {execution.orderId}, Shares: {execution.shares}, LastLiquidity: {execution.lastLiquidity}")

# Example usage
bot = GridTradingBot("AAPL", 1.0, 10)
bot.start()



ERROR -1 502 Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002
